## From a list of named entities to Gremlin graph. 



This notebook take about 11 min to complete. 

### Dataset

In the previous notebook, pdf documents were converted to csv files using OCR and NER tool. 
We are going to use the csv files to build a Gremlin compatible graph. 

There are 8,995 CSV files in S3.  

In [17]:
bucket_name = 'hcls-kg-workshop'
prefix = 'stdized-data-new/comprehend_results/csv'
prefix_output = prefix.replace('csv','gremlin_data')
prefix_output

'stdized-data-new/comprehend_results/gremlin_data'

In [2]:
!aws s3 ls s3://{bucket_name}/{prefix}/

2021-09-01 06:26:56       3851 10000.csv
2021-09-01 01:25:17       3768 10002.csv
2021-09-01 02:51:09       6463 10003.csv
2021-09-01 05:51:56       3139 10007.csv
2021-09-01 04:53:59      12281 10010.csv
2021-09-01 05:28:27       5689 10011.csv
2021-09-01 02:39:30       7174 10012.csv
2021-09-01 04:02:22       1464 10015.csv
2021-09-01 02:01:44       5059 10016.csv
2021-09-01 05:03:52       3176 10017.csv
2021-09-01 06:24:06       4670 10021.csv
2021-09-01 02:18:30       1652 10022.csv
2021-09-01 05:58:27       6899 10024.csv
2021-09-01 05:45:13       1792 10025.csv
2021-09-01 01:51:23       6539 10034.csv
2021-09-01 05:42:29       4821 10035.csv
2021-09-01 03:47:25       3185 1004.csv
2021-09-01 03:49:26       2751 10050.csv
2021-09-01 02:05:44       2862 10058.csv
2021-09-01 01:37:17       3111 10059.csv
2021-09-01 04:59:18       4042 10061.csv
2021-09-01 02:45:07       4909 10062.csv
2021-09-01 02:23:21       3746 10063.csv
2021-09-01 05:39:20       4091 10064.csv
2021-09-01 06:15:

In [3]:
#cp ../v1 . -r

### SageMaker Processing

SageMaker Processing is used for this batch process. 
The input is a cvs file. The output is also a csv file that describes Gremlin compatible graph. 

Gremlin graph requires two types of csv files. The first type 

### Modules

In [4]:
import boto3
import pandas as pd
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

my_session = boto3.session.Session()
my_region = my_session.region_name
my_account = boto3.client('sts').get_caller_identity().get('Account')

In [5]:
my_region, my_account

('us-east-2', '287351682664')

In [6]:
repo_name = 'ner2gremlin'
#bucket_name = bucket_name_small
#prefix = prefix_small

ner2gremlin.py is the entry point script. 

In [7]:
!cat ner2gremlin.py

import argparse
import glob 
import time
import logging
import subprocess
import os

## Config
from pathlib2 import Path
from datetime import datetime
import src
from src.knowledge_graph_transformer import knowledgeGraphTransformer
from src.utils import list_csvs
import config

#
import boto3
s3r = boto3.resource('s3')

#

INPUT_DIR = "/opt/ml/processing/input" # change this for testing locally
OUTPUT_DIR = "/opt/ml/processing/output" # change this for testing locally 
MODEL_DIR = "/opt/ml/model" # change this for testing locally
SOURCE_DIR = "/opt/ml/code" # change this for testing locally

logging.basicConfig(level=logging.DEBUG)
logging.info(subprocess.call(f'ls -lR {INPUT_DIR}'.split()))
logging.info(subprocess.call(f'ls -lR {SOURCE_DIR}'.split()))

print('passed')
############################
# helper functions
############################
def get_host_id():
    host = os.environ['HOSTNAME']
    id_ = "".join(host.split('.')[0].split('-')[-2:])
    return id_

####################

### Docker file

In [8]:
docker_str = \
f"""
FROM nvidia/cuda:10.2-runtime-ubuntu18.04

RUN apt-get update \
  && apt-get -y install --no-install-recommends git python3 python3-pip libsm6 libxext6 \
    libxrender1 libglib2.0-0 python3-setuptools \
    ffmpeg libgl1-mesa-glx \
  && rm -rf /var/lib/apt/lists/*

RUN pip3 install --upgrade pip

RUN pip3 install tqdm ConfigParser pathlib2 pandas boto3

WORKDIR /opt/ml/code
COPY ner2gremlin.py .
COPY ./v1/config.py .

WORKDIR /opt/ml/code/src
COPY ./v1/src/knowledge_graph_transformer.py .
COPY ./v1/src/utils.py .
COPY ./v1/src/__init__.py .

WORKDIR /opt/ml/code/src/base
COPY ./v1/src/base/base_graph_object_collections.py .
COPY ./v1/src/base/base_graph_objects.py .

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONPATH=/opt/ml/code
ENV MY_REGION={my_region}
ENV BUCKET_NAME={bucket_name}
ENV S3_PREFIX={prefix_output}

ENV NVIDIA_VISIBLE_DEVICES all
ENV NVIDIA_DRIVER_CAPABILITIES compute,video,utility

ENTRYPOINT ["python3", "/opt/ml/code/ner2gremlin.py"]

"""

### Build a docker image

In [9]:
with open("Dockerfile.part2", "w") as f:
    f.write(docker_str)
    
!docker build -t {repo_name} -f Dockerfile.part2 .

Sending build context to Docker daemon  3.856MB
Step 1/22 : FROM nvidia/cuda:10.2-runtime-ubuntu18.04
 ---> 341bf28bafec
Step 2/22 : RUN apt-get update   && apt-get -y install --no-install-recommends git python3 python3-pip libsm6 libxext6     libxrender1 libglib2.0-0 python3-setuptools     ffmpeg libgl1-mesa-glx   && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 885a4d5d3e55
Step 3/22 : RUN pip3 install --upgrade pip
 ---> Using cache
 ---> 844b5275cee1
Step 4/22 : RUN pip3 install tqdm ConfigParser pathlib2 pandas boto3
 ---> Using cache
 ---> 16b29386f9e4
Step 5/22 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 574e8ec2eda0
Step 6/22 : COPY ner2gremlin.py .
 ---> Using cache
 ---> 86005db116a8
Step 7/22 : COPY ./v1/config.py .
 ---> Using cache
 ---> 7ca9bc71040b
Step 8/22 : WORKDIR /opt/ml/code/src
 ---> Using cache
 ---> f062aea2a85e
Step 9/22 : COPY ./v1/src/knowledge_graph_transformer.py .
 ---> Using cache
 ---> 8d20a47685ba
Step 10/22 : COPY ./v1/src/utils.py .
 ---> Usi

### Push to ECR

In [10]:
%%bash -s {my_account} {my_region} {repo_name}

# If the repository doesn't exist in ECR, create it.
$(aws ecr get-login --no-include-email --region $2)

aws ecr describe-repositories --region $2 --repository-names $3 > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --region $2 --repository-name $3 > /dev/null
fi

docker tag $3:latest $1.dkr.ecr.$2.amazonaws.com/$3:latest
docker push $1.dkr.ecr.$2.amazonaws.com/$3:latest

Login Succeeded
The push refers to repository [287351682664.dkr.ecr.us-east-2.amazonaws.com/ner2gremlin]
f438db3fc386: Preparing
316c458f3d71: Preparing
7f65dc05765d: Preparing
1bcff25f135a: Preparing
73d51cb2787f: Preparing
e9dbd9641e84: Preparing
af0cb8c95bb3: Preparing
540046600a0e: Preparing
3da1016318c1: Preparing
30d66d5d6a48: Preparing
bd78eccb511f: Preparing
e007c5e2a0ac: Preparing
e517772ca3d8: Preparing
ea8773aeab99: Preparing
7f31bfc76a5e: Preparing
7c4b35193021: Preparing
af37baa7f084: Preparing
2bd88e8e53d8: Preparing
8c042ee1e25b: Preparing
21639b09744f: Preparing
e9dbd9641e84: Waiting
af0cb8c95bb3: Waiting
540046600a0e: Waiting
3da1016318c1: Waiting
30d66d5d6a48: Waiting
bd78eccb511f: Waiting
e007c5e2a0ac: Waiting
e517772ca3d8: Waiting
ea8773aeab99: Waiting
7f31bfc76a5e: Waiting
7c4b35193021: Waiting
af37baa7f084: Waiting
2bd88e8e53d8: Waiting
8c042ee1e25b: Waiting
21639b09744f: Waiting
316c458f3d71: Layer already exists
1bcff25f135a: Layer already exists
73d51cb2787f: L

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Run a SageMaker Processing job

In [11]:
def run(instance_type, instance_count):
    processor = Processor(image_uri=f'{my_account}.dkr.ecr.{my_region}.amazonaws.com/{repo_name}:latest',
                                       role=get_execution_role(),
                                       instance_count=instance_count,
                                       base_job_name=f"{repo_name}-job",
                                       instance_type=instance_type)

        
    destination = '/opt/ml/processing/input/'
    pi = ProcessingInput(
                    source=f's3://{bucket_name}/{prefix}/', 
                    s3_data_type='S3Prefix',
                    s3_input_mode='File',
                    s3_data_distribution_type='ShardedByS3Key',
                    destination=destination)
        
    
    
    processor.run(inputs=[pi],
                  outputs=[ProcessingOutput(source='/opt/ml/processing/output', 
                        destination=f's3://{bucket_name}/{prefix_output}/',
                        output_name='output',
                        s3_upload_mode='Continuous')],
                  wait=False,
                  logs=False)
    
    return processor

run('ml.m5.4xlarge', 1)


Job Name:  ner2gremlin-job-2021-09-14-16-34-36-856
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://hcls-kg-workshop/stdized-data-new/comprehend_results/csv/', 'LocalPath': '/opt/ml/processing/input/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://hcls-kg-workshop/stdized-data-new/comprehend_results/gremlin_data_09142021/', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'Continuous'}}]


### Output csv file

It should be noted that some nodes and edges share the same names (e.g. SYSTEM_ORGAN_SITE). This is due to Amazon Comprehend Medical covention. Type of entity and RelationshipType between two entities share the same name.  

 

In [12]:
!aws s3 ls s3://{bucket_name}/{prefix_output}/

                           PRE edges/
                           PRE nodes/


In [13]:
!aws s3 ls s3://{bucket_name}/{prefix_output}/edges/

2021-09-14 16:43:25     129426 ACUITY_edges.csv
2021-09-14 16:43:25      63072 DIRECTION_edges.csv
2021-09-14 16:43:25     387680 DOSAGE_edges.csv
2021-09-14 16:43:25      32390 DURATION_edges.csv
2021-09-14 16:43:25      90420 FORM_edges.csv
2021-09-14 16:43:25      47651 FREQUENCY_edges.csv
2021-09-14 16:43:25     226034 ROUTE_OR_MODE_edges.csv
2021-09-14 16:43:25      53366 STRENGTH_edges.csv
2021-09-14 16:43:25     261806 SYSTEM_ORGAN_SITE_edges.csv
2021-09-14 16:43:25     272291 TEST_UNIT_edges.csv
2021-09-14 16:43:25    2021579 TEST_VALUE_edges.csv
2021-09-14 16:43:25   10171497 doc2text_edges.csv


In [18]:
!aws s3 ls s3://{bucket_name}/{prefix_output}/nodes/

2021-08-26 17:06:04       1059 ACUITY_nodes.csv
2021-08-26 17:06:05      11651 ADDRESS_nodes.csv
2021-08-26 17:06:04       3018 AGE_nodes.csv
2021-08-31 00:29:57     160257 ANATOMY_nodes.csv
2021-08-26 17:06:04      20070 BRAND_NAME_nodes.csv
2021-08-26 17:06:04       8558 DATE_nodes.csv
2021-08-26 17:06:05       5185 DIRECTION_nodes.csv
2021-08-26 17:06:05      56723 DOSAGE_nodes.csv
2021-08-26 17:06:05       6150 DURATION_nodes.csv
2021-08-26 17:06:05     449267 DX_NAME_nodes.csv
2021-08-26 17:06:05       2650 FORM_nodes.csv
2021-08-26 17:06:05       4396 FREQUENCY_nodes.csv
2021-08-26 17:06:05     717851 GENERIC_NAME_nodes.csv
2021-08-26 17:06:05        407 ID_nodes.csv
2021-08-31 00:29:57     576657 MEDICAL_CONDITION_nodes.csv
2021-08-31 00:29:57     956615 MEDICATION_nodes.csv
2021-08-26 17:06:04      17896 NAME_nodes.csv
2021-08-26 17:06:05       1347 PHONE_OR_FAX_nodes.csv
2021-08-26 17:06:05      43810 PROCEDURE_NAME_nodes.csv
2021-08-26 17:06:05        147 PROFESSION_nodes.csv

In [19]:
!aws s3 cp s3://{bucket_name}/{prefix_output}/edges/doc2text_edges.csv .

download: s3://hcls-kg-workshop/stdized-data-new/comprehend_results/gremlin_data/edges/doc2text_edges.csv to ./doc2text_edges.csv


In [20]:
pd.read_csv('doc2text_edges.csv')

,~from,~to,~id,~label,Id:String,BeginOffset:String,EndOffset:String,Score:String,Name:String,Type:String,Traits:String,Attributes:String,RelationshipScore:String,RelationshipType:String,distId:String
0,a5233091-8f33-4698-82a3-ea1bedad9262,597f6b28-48f6-437f-8617-a99f46f8bcd7,1b7ddfff-6409-44d3-ac5f-55eac779936f,contains_text,2,582,593,0.701431,LPS selects,TEST_NAME,[],"[{'Type': 'TEST_VALUE', 'Score': 0.46526592969...",NaN,NaN,NaN
1,a5233091-8f33-4698-82a3-ea1bedad9262,ded29b86-efb6-46a2-9141-929d330b6bf3,a5837157-0f2d-486f-9782-dd3606634bff,contains_text,5,725,734,0.532359,increased,TEST_VALUE,[],NaN,NaN,NaN,NaN
2,b5176f2c-f3c5-4c6a-a11f-5e4cd03fe3b2,fa4ba2fc-4735-4e26-8d60-d2c93805851f,9c423a88-9ee3-4e0f-b8cd-4d1c136b9691,contains_text,0,200,206,0.859620,testis,SYSTEM_ORGAN_SITE,[],NaN,NaN,NaN,NaN
3,b5176f2c-f3c5-4c6a-a11f-5e4cd03fe3b2,5a774b5d-2b4d-4e64-bf91-1fbefbed9e74,8097534e-ed6b-4a8a-8860-10ae15631ba9,contains_text,2,38,58,0.917958,testicular neoplasia,DX_NAME,"[{'Name': 'DIAGNOSIS', 'Score': 0.908835589885...",NaN,NaN,NaN,NaN
4,b5176f2c-f3c5-4c6a-a11f-5e4cd03fe3b2,83b68510-59f4-4a00-9c68-3aa82f79958c,ecb2d118-9b4e-4300-bbba-88d499608579,contains_text,3,188,206,0.856163,undescended testis,DX_NAME,"[{'Name': 'DIAGNOSIS', 'Score': 0.786847412586...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37782,0829069e-641d-427a-ac39-fbe250468cc2,a4744601-ffe7-42c7-baef-0d51cf5fa49d,8835e235-6db5-40d7-891b-707bd994a969,contains_text,2,72,102,0.791281,acetylpyruvic acid ethyl ester,GENERIC_NAME,[],"[{'Type': 'ROUTE_OR_MODE', 'Score': 0.73218286...",NaN,NaN,NaN
37783,19add4bb-71c7-4b79-8c58-08baec53484d,2f471b4b-f6e7-44ff-ad01-38f5ed50e11b,cb91c50a-8ad0-45c2-9b61-12c2925c36c2,contains_text,3,100,114,0.956091,physically ill,DX_NAME,"[{'Name': 'DIAGNOSIS', 'Score': 0.585113286972...",NaN,NaN,NaN,NaN
37784,9984b3b2-b0a2-43f0-a2ad-51c447d5c627,4d8ecef0-f9ca-4470-a608-88c39d612334,803d42dc-4737-4d90-9c19-dd447cb8bcfa,contains_text,4,117,128,0.707440,trichinosis,DX_NAME,"[{'Name': 'DIAGNOSIS', 'Score': 0.669039309024...","[{'Type': 'ACUITY', 'Score': 0.664781033992767...",NaN,NaN,NaN
37785,9984b3b2-b0a2-43f0-a2ad-51c447d5c627,b3639659-14a7-47e9-a105-0471591c3c95,2266f305-77dc-4ac3-896d-2ca2f813d33b,contains_text,7,238,249,0.883047,eosinopenia,DX_NAME,"[{'Name': 'DIAGNOSIS', 'Score': 0.844857692718...",NaN,NaN,NaN,NaN


### Note

Amazon Comprehend Medical has another node feature "Category". If Category is used as the node type, it is able to avoid the confusion where node and edge share a same name. We have tested the case where Category was used as the node type. 
It did not work well for the next Graph ML step. We expect that is due to the fact Category feature has less granularity than Type feature. 